In [1]:
%load_ext autoreload
%autoreload 2

# Import Libraries

In [2]:
import os

import pandas as pd
import numpy as np
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder


nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/deniskirbaba/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Read Data

In [3]:
from pathlib import Path

path = Path("../data/")
train = pd.read_csv(Path(path, "train.csv"))
test = pd.read_csv(Path(path, "test.csv"))
print("Number of rows and columns in the train data set:", train.shape)
print("Number of rows and columns in the test data set:", test.shape)
train.head()

Number of rows and columns in the train data set: (48665, 2)
Number of rows and columns in the test data set: (12167, 2)


,rate,text
0,4,Очень понравилось. Были в начале марта с соба...
1,5,В целом магазин устраивает.\nАссортимент позво...
2,5,"Очень хорошо что открылась 5 ка, теперь не над..."
3,3,Пятёрочка громко объявила о том как она заботи...
4,3,"Тесно, вечная сутолока, между рядами трудно ра..."


# Label encoding

In [6]:
le = LabelEncoder()

train.rate = le.fit_transform(train.rate)
train.head()

,rate,text
0,3,Очень понравилось. Были в начале марта с соба...
1,4,В целом магазин устраивает.\nАссортимент позво...
2,4,"Очень хорошо что открылась 5 ка, теперь не над..."
3,2,Пятёрочка громко объявила о том как она заботи...
4,2,"Тесно, вечная сутолока, между рядами трудно ра..."


# Text Preprocessing

In [8]:
stopwords = nltk.corpus.stopwords.words("russian")

# Init tf-idf
vect_word = TfidfVectorizer(
    max_features=100,
    lowercase=True,
    analyzer="word",
    stop_words=stopwords,
    ngram_range=(1, 3),
    dtype=np.float32
)

In [9]:
# Train tf-idf
X_train = vect_word.fit_transform(train["text"])
# Map tf-idf on test
X_test = vect_word.transform(test["text"])

y_train = train["rate"]

In [10]:
X_train.shape, y_train.shape

((48665, 100), (48665,))

# Init Model

In [11]:
# Init logreg model
logreg = LogisticRegression(
    C=2,
    random_state=42
)

# Train Model

In [12]:
# Train logreg
logreg.fit(X_train, y_train)

LogisticRegression(C=2, random_state=42)

# Create Predicts

In [13]:
# Predict probabilities
preds_proba = logreg.predict_proba(X_test)
# Get classes
preds = np.argmax(preds_proba, axis=1)
# pred_labels = le.inverse_transform(preds)

# Create submission

In [10]:
sample_submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))
sample_submission["rate"] = preds
sample_submission.rate = le.inverse_transform(sample_submission.rate)
sample_submission.head()

,rate
0,5
1,5
2,5
3,3
4,5


In [11]:
sample_submission.to_csv("submission.csv", index=False)